# Hardware Testing Processes :

- The instrumentation course covers the base topics of industrial instrumentation: sensors and how they work, A-D and D-A converters, instruments useful for taking measures, communication buses used in instrumentation, calibration, …
- The instrumentation labs make use of the python and VISA protocol. It allows the students to learn a software widely used in the industry in order to design automated and autonomous instrumentation systems.

## Setup:

Create a new -venv :

```>>> python -m venv HTP_venv ```

### requirements.txt :
```
numpy
matplotlib

ipykernel
jupyter
```


Activate the venv and install dependencies :
```
>>> .\HTP_venv\Scripts\Activate.ps1
>>> pip install -r requirements.txt
```

Register our venv as a Jupyter kernel:
```
>>> python -m ipykernel install --user --name=HTP_venv --display-name="Python (HTP_venv)"
>>> jupyter kernelspec list
(should show) : htp_venv         C:\Users\user\AppData\Roaming\jupyter\kernels\htp_venv
```

### Drivers (Windows 10):

DG1022 : 
- https://www.rigolna.com/download/ 
- Filter by DG1000
- then Download > UltraSigma Instrument Connectivity Driver

DS1104 :


## Session 1- the function generator- Rigol DG1022

### 1 - Connect to DG1022

In [1]:
# 1. Find the unique ID of instrument :
import pyvisa
import time

def connect_to_dg1022(verbose=False):
    try:
        # Create resource manager
        rm = pyvisa.ResourceManager()
        resources = rm.list_resources()
        if verbose : print(f"PyVisa Resources detected: {resources}")
        
        # Find DG1022 automatically
        dg1022_id = None
        for resource in resources:
            if "DG1" in resource or "0x1AB1" in resource:
                dg1022_id = resource
                break
        
        if not dg1022_id:
            print("DG1022 not found!")
            return None, None
        
        if verbose : print(f"Found DG1022 ID: {dg1022_id}")
        
        # Connect to device
        dg = rm.open_resource(dg1022_id)
        
        # Configure communication settings
        dg.timeout = 10000
        dg.read_termination = '\n'
        dg.write_termination = '\n'
        dg.chunk_size = 20480
        dg.clear()
        
        time.sleep(0.5)
        
        # Test communication
        dg.write('*IDN?')
        time.sleep(0.5)
        idn = dg.read()
        print(f"DG1022 Connected! Device Info: {idn}")
        
        return rm, dg
        
    except Exception as e:
        print(f"Connection failed: {e}")
        if 'rm' in locals():
            rm.close()
        return None, None

def close_dg1022(rm,dg):
    dg.close()
    rm.close()
    print("Connections closed.")

# ================ EXECUTION ===========================
rm, dg = connect_to_dg1022(True)

if dg:
    print("Device ready for commands!")
    
    #! CLOSE
    close_dg1022(rm, dg)
else:
    print("Failed to connect to DG1022")


# Challenge, make it automatic to connect one function generator and one oscilloscope with the right ID and
    #  variable. 1 bonus point out of 20 will be provided if you can do it before the last session, only to the first one to
    #  achieve it.


PyVisa Resources detected: ('USB0::0x1AB1::0x0588::DG1D114703832::INSTR', 'ASRL3::INSTR', 'ASRL4::INSTR', 'ASRL5::INSTR', 'ASRL6::INSTR', 'ASRL11::INSTR')
Found DG1022 ID: USB0::0x1AB1::0x0588::DG1D114703832::INSTR
DG1022 Connected! Device Info: RIGOL TECHNOLOGIES,DG1022 ,DG1D114703832,,00.02.00.06.00.02.05
Device ready for commands!
Connections closed.


### 2 - Toggle CH1

In [ ]:
# 2. To start create a simple code that will open and close Channel 1 ten times with a delay of one second between 
# the steps (import time to help you). To communicate correctly, it is important to open the resource manager with 
# the ID of your device and create your instrument in the code. To do this step, use the function open_resources(ID) 
# on the variable of your resource manager and store it in a variable that will correspond to your instrument 
# (example : osci = rm.open_resource(ID)).  Do not forget to close the communication  with your instrument and the resource 
# manager after the execution of your code (For all the functions used by the resource manager, you can get more information on 
# this web page https://pyvisa.readthedocs.io/en/latest/api/resources.htm

import time

rm, dg = connect_to_dg1022()

try :
    print("Channel 1 and 2 toggle tests....")
    for i in range(10):
        dg.write('OUTPut:CH2 ON ')
        # time.sleep(0.05)
        # dg.write('OUTPut:CH1 OFF ')
        time.sleep(1)

        dg.write('OUTPut:CH2 OFF ')
        # time.sleep(0.05)
        # dg.write('OUTPut:CH1 ON ')
        time.sleep(1)
    print("Test done ! Channel 2 successfully toggled 10 times.")
except Exception as e:
        print(f"Connection failed: {e}")

# Close
close_dg1022(rm, dg)





DG1022 Connected! Device Info: RIGOL TECHNOLOGIES,DG1022 ,DG1D114703832,,00.02.00.06.00.02.05
Channel 1 and 2 toggle tests....
Test done ! Channel 1 successfully toggled 10 times.
Connections closed.


### 3 - Sine Function

In [ ]:
# 3. based on the programming Manual, explore the functionalities in order to create a sine of
#  5V on channel1 and a square of 3volt on channel2.To visualize the generated waves,you
#  can use the oscilloscope and BNC wires in the lab.
import time

#? Source: Example 7 in DG1000 Documentation

rm, dg = connect_to_dg1022()

# Channel 2:
dg.write('VOLT:UNIT VPP')           # Set the amplitude unit of CH1 
time.sleep(0.05)
dg.write('APPL:SQUare 1000,2.5,0.5 ')  # Set the frequency, amplitude and offset of sine wave output from CH1 
time.sleep(0.05)

dg.write('PHAS 10 ')                # Set the initial phase of wave output from CH1 
time.sleep(0.05)

# dg.write('OUTPut:CH2 ON ')                      # Enable the [Output] connector of CH1 at the front panel 
ch = "2"
dg.write(f'OUTPut:{ch} OFF ') 

#TODO: f-string problem comes from trailing space after "ON " ?


# Close
# time.sleep(20)
close_dg1022(rm, dg)

DG1022 Connected! Device Info: RIGOL TECHNOLOGIES,DG1022 ,DG1D114703832,,00.02.00.06.00.02.05
Connections closed.


In [16]:
def generate_waveform(channel, waveform_type, frequency, amplitude, offset=0, phase=0, duty_cycle=50):
    try:
        # Apply waveform
        dg.write(f'SOUR{channel}:APPL:{waveform_type} {frequency},{amplitude},{offset}')
        time.sleep(0.05)
        
        # PHASE
        if phase != 0: dg.write(f'SOUR{channel}:PHAS {phase}')
        time.sleep(0.05)
        
        # Duty cycle for square/pulse waves
        if waveform_type in ['SQU', 'PULS'] and duty_cycle != 50:
            dg.write(f'SOUR{channel}:FUNC:{waveform_type}:DCYC {duty_cycle}')
            time.sleep(0.05)
        
        # Enable output
        dg.write(f'OUTPut:{channel} ON')
        time.sleep(0.05)

        print(f"CH{channel}: {waveform_type} {amplitude}V @ {frequency}Hz - ON")
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False

# =========================================
rm, dg = connect_to_dg1022()

# 5V sine on CH1
# generate_waveform(channel=1, waveform_type='SIN', frequency=1000, amplitude=5)

# 3V square on CH2
generate_waveform(channel=2, waveform_type='SQUare', frequency=1000, amplitude=3)


# Close
# time.sleep(100)
close_dg1022(rm, dg)

DG1022 Connected! Device Info: RIGOL TECHNOLOGIES,DG1022 ,DG1D114703832,,00.02.00.06.00.02.05
CH2: SQUare 3V @ 1000Hz - ON
Connections closed.


### 4 -

In [9]:
# 4.Onabreadboard,useacapacitortoflattenanexponentialwavegeneratedbythegenerator
#  (findtheright informationinthemanual).Watchyoursignalontheoscilloscope.




### 5 - 

In [ ]:
#  5. Tobecomplete,createasquaresignalonchannel1thathas
#  —afrequencyof1250Hz
#  —anoffsetof2.3V
#  —aduty-cycleof60%
#  —anamplitudechangingevery2secondsform2.3Vto4.6V

### 6 - 

In [ ]:
# . Record all the information of exercise 5 in a CSV file with a timeline and show it with a
#  graph

### 7 - 

In [ ]:
# 7. Challenge : Use a song as an arbitrary waveform and make it generated by the generator
#  into the speaker. (ask the speaker to the professors)

---
## Session 2- oscilloscope :